In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
import calendar
warnings.simplefilter(action = 'ignore', category = FutureWarning)
from time import sleep

In [17]:
from helpers.scrape import get_first_basket
from helpers.utils import getId


def get_monthly_games(month_url) :

    url = f'https://www.basketball-reference.com/{month_url}'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    table = soup.find('table')
    while table.find_all('tr', class_ = 'thead') :
        table.find('tr', class_ = 'thead').decompose()
    games = pd.read_html(str(table))[0].rename(columns = {'Start (ET)': 'Time'})
    games['Date'] = pd.to_datetime(games['Date'])
    games['Home'] = [x['href'].split('/')[2] for x in table.find_all('a', href = True) if 'teams' in x['href']][1::2]
    games['Away'] = [x['href'].split('/')[2] for x in table.find_all('a', href = True) if 'teams' in x['href']][0::2]
    games['game_id'] = [getId(x) for x in table.find_all('a', href = True) if 'boxscores' in x['href']][1::2]

    return games[['game_id', 'Date', 'Time', 'Home', 'Away']]

In [18]:
months = list(calendar.month_name)[1:]
seasons = [2018]

for season in seasons :
    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
    page = requests.get(url)
    if page.status_code == 429 :
        raise ValueError('Rate limited...')
    soup = BeautifulSoup(page.content, 'lxml')
    month_urls = [x['href'] for x in soup.find_all('a', href = True) if 'games' in x['href'] 
                    and any(m.lower() in x['href'] for m in months)]

    for m, month_url in enumerate(month_urls) :

        games_monthly = get_monthly_games(month_url)
        games_monthly['season'] = season

        first_basket_info = []
        for i, gameId in enumerate(games_monthly['game_id'])  :

            print(f'[{round(100*(i+1)/len(games_monthly))}%...] season :  {season-1}-{season}, month :  {month_url.split("-")[-1].split(".")[0]} ({gameId})')
            sleep(5)
            first_basket_info.append(get_first_basket(gameId))


        first_basket_df = pd.concat(first_basket_info)
        first_basket_df = games_monthly.merge(first_basket_df, on = ['game_id', 'Home', 'Away'], how = 'inner')
        if m == 0 :
            first_basket_df.to_csv(f'data/first_basket_{season}.csv', index = False)
        else :
            first_basket_df.to_csv(f'data/first_basket_{season}.csv', mode = 'a', header = False, index = False)

[1%...] season :  2017-2018, month :  october (201710170CLE)
[2%...] season :  2017-2018, month :  october (201710170GSW)
[3%...] season :  2017-2018, month :  october (201710180DET)
[4%...] season :  2017-2018, month :  october (201710180IND)
[5%...] season :  2017-2018, month :  october (201710180ORL)
[6%...] season :  2017-2018, month :  october (201710180WAS)
[7%...] season :  2017-2018, month :  october (201710180BOS)
[8%...] season :  2017-2018, month :  october (201710180MEM)
[9%...] season :  2017-2018, month :  october (201710180DAL)
[10%...] season :  2017-2018, month :  october (201710180UTA)
[11%...] season :  2017-2018, month :  october (201710180SAS)
[12%...] season :  2017-2018, month :  october (201710180PHO)
[12%...] season :  2017-2018, month :  october (201710180SAC)
[13%...] season :  2017-2018, month :  october (201710190TOR)
[14%...] season :  2017-2018, month :  october (201710190OKC)
[15%...] season :  2017-2018, month :  october (201710190LAL)
[16%...] season :

ValueError: not enough values to unpack (expected 3, got 2)

In [19]:
gameId

'201710220LAL'